In [1]:
import re
import pandas as pd
from pyvi import ViTokenizer, ViPosTagger


In [2]:
#none
text = '''
Luật số 22/2023/qh15 ngày 23 tháng 06 năm 2023 của quốc hội
các gói thầu thuộc dự án đầu tư của doanh nghiệp nhà nước theo quy định của Luật doanh nghiệp và doanh nghiệp do doanh nghiệp nhà nước nắm giữ 100% vốn điều lệ
'''
#DC Luật đầu tư công 
text_1 = '''Luật số 22/2023/qh15 ngày 23 tháng 06 năm 2023 của quốc hội
gói thầu thuộc dự toán mua sắm có giá gói thầu trong hạn mức từ trên 50 triệu đồng đến 100 triệu đồng ;
gói thầu thuộc dự án đầu tư của doanh nghiệp nhà nước , doanh nghiệp do doanh nghiệp nhà nước nắm giữ 100 % vốn điều lệ 
và dự án đầu tư theo Luật đầu tư công , bao gồm : gói thầu cung cấp dịch vụ tư vấn có giá gói thầu trong hạn mức không quá 500 triệu đồng , 
gói thầu phi tư vấn , hàng hoá , xây lắp , hỗn hợp có giá gói thầu trong hạn mức không quá 01 tỉ đồng .trường hợp cần điều chỉnh hạn mức quy định tại 
điểm này để phù hợp với điều kiện kinh tế-xã hội và yêu cầu quản lí nhà nước , chính phủ trình uỷ ban thường vụ quốc hội xem xét , Quyết định .
'''
# CC
text_2 = '''Thông tư số 88/2015/tt-btc ngày 09 tháng 06 năm 2015 của bộ tài chính
Nghị định số 24/2006/nđ-cp ngày 06/3/2006 của chính phủ quy định chi tiết thi hành pháp Lệnh phí và lệ phí
căn cứ vào Hiến pháp 2013 thực hiện các biện pháp sau.
'''
# CC
text_3 = ' căn cứ Nghị định số 57/2002/nđ-cp ngày 03/6/2002'
#none
text_4 = 'thông tư số 92/2015/tt-btc ngày 15 tháng 06 năm 2015 của bộ tài chính' + ' bãi bỏ điểm d khoản 6 điều 2 Thông tư số 111/2013/t-btc hướng dẫn về thuế thu nhập cá nhân đối với cá nhân có thu nhập từ trúng thưởng trong casino'
# DaC Luật quản_lí thuế tính
text_5 = 'Luật số 31/2024/qh15 ngày 18 tháng 01 năm 2024 của quốc hội trường hợp cơ quan , đơn vị , tổ chức' + ' thực hiện bồi thường chậm chi trả thì khi thanh toán tiền bồi thường , hỗ trợ cho người có đất thu hồi , chủ sở hữu tài sản , ngoài tiền bồi thường , hỗ trợ theo phương án bồi thường , hỗ trợ , tái định cư được cấp có thẩm quyền phê duyệt thì người có đất thu hồi , chủ sở hữu tài sản còn được thanh toán thêm một khoản tiền bằng mức tiền chậm nộp theo quy định của Luật quản lí thuế tính trên số tiền chậm trả và thời gian chậm trả'
# Dac
text_6 = 'Luật Sửa đổi, bổ sung một số điều của Luật thi hành án dân sự Đối với người phải thi hành án là người bị kết án thuộc trường hợp quy định tại điểm c mục 3 Nghị quyết số 32/1999/QH10 của Quốc hội thì được miễn nghĩa vụ thi hành án phí trong bản án hình sự mà người đó phải chấp hành .'
# CC
text_7 = 'Luật Sửa đổi, bổ sung một số điều của Luật thi hành án dân sự căn cứ Hiến pháp nước cộng hòa xã hội chủ nghĩa việt nam.'



In [3]:
import json
import string
import enchant

class tvpl_function:
    def __init__(self):
        self.pattern = 'aàảãáạăằẳẵắặâầẩẫấậeèẻẽéẹêềểễếệiìỉĩíịoòỏõóọôồổỗốộơớờởỡợuùủũúụưừửữứựyỳỷỹýỵđ'

        self.words_en = enchant.Dict("en_US")
        
        self.specialCharset = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~'

        self.words = []
        with open("words.txt", "r", encoding='utf-8') as file:
            for f in file:
                word = json.loads(f)
                self.words.append(word["text"])

        self.syllables = []
        with open("syllable.txt", "r", encoding="utf-8") as file:
            for line in file:
                self.syllables.append(line.strip())
                
        self.mapping={
            'óa':'oá', 'òa':'oà', 'ỏa':'oả', 'õa':'oã', 'ọa':'oạ',
            'óe':'oé', 'òe':'oè', 'ỏe':'oẻ', 'õe':'oẽ', 'ọe':'oẹ',
            'úy':'uý', 'ùy':'uỳ', 'ủy':'uỷ', 'ũy':'uỹ', 'ụy':'uỵ',
            'úâ':'uấ', 'ùâ':'uầ', 'ủâ':'uẩ', 'ũâ':'uẫ', 'ụâ':'uậ',
            'úe':'ué', 'ùe':'uè', 'ủe':'uẻ', 'ũe':'uẽ', 'ụe':'uẹ',
            'úê':'uế', 'ùê':'uề', 'ủê':'uể', 'ũê':'uễ', 'ụê':'uệ',
            'úơ':'uớ', 'ùơ':'uờ', 'ủơ':'uở', 'ũơ':'uỡ', 'ụơ':'uợ',
            'úô':'uố', 'ùô':'uồ', 'ủô':'uổ', 'ũô':'uỗ', 'ụô':'uộ',
            'iá':'ía', 'ià':'ìa', 'iả':'ỉa', 'iã':'ĩa', 'iạ':'ịa',
            'yá':'ýa', 'yà':'ỳa', 'yả':'ỷa', 'yã':'ỹa', 'yạ':'ỵa',
            'uá':'úa', 'uà':'ùa', 'uả':'ủa', 'uã':'ũa', 'uạ':'ụa',
            'ưá':'ứa', 'ưà':'ừa', 'ưả':'ửa', 'ưã':'ữa', 'ưạ':'ựa',
            'ứơ':'ướ', 'ừơ':'ườ', 'ửơ':'ưở', 'ữơ':'ưỡ', 'ựơ':'ượ',
            'íê':'iế', 'ìê':'iề', 'ỉê':'iể', 'ĩê':'iễ', 'ịê':'iệ',
            'ýê':'yế', 'ỳê':'yề', 'ỷê':'yể', 'ỹê':'yễ', 'ỵê':'yệ',
            'uí':'úi', 'uì':'ùi', 'uỉ':'ủi', 'uĩ':'ũi', 'uị':'ụi',
            'aó':'áo', 'aò':'ào', 'aỏ':'ảo', 'aõ':'ão', 'aọ':'ạo',
            'qúa':'quá', 'qùa':'quà', 'qủa':'quả', 'qũa':'quã', 'qụa': 'quạ',
            'Qúa':'Quá', 'Qùa':'Quà', 'Qủa':'Quả', 'Qũa':'Quã', 'Qụa': 'Quạ', 
            'gía':'giá', 'gìa':'già', 'gỉa':'giả', 'gĩa':'giã', 'gịa': 'giạ',
            'Gía':'Giá', 'Gìa':'Già', 'Gỉa':'Giả', 'Gĩa':'Giã', 'Gịa': 'Giạ',
        }
        
        self.type_documents =['Hiến_pháp', 'Bộ_luật', 'Luật', 'Pháp_lệnh', 'Lệnh', 'Nghị_quyết', 'Nghị_quyết liên_tịch', 'Nghị_định', 'Quyết_định', 'Thông_tư', 'Thông_tư liên_tịch', 'Chỉ_thị']
        

    # chuẩn hóa bảng mã tiếng việt
    # Tạo ra từ điện bộ kí tự
    def load_DictChar(self):
        dic = {}
        char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
            '|')
        charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
            '|')
        for i in range(len(char1252)):
            dic[char1252[i]] = charutf8[i]
        return dic
    
    # Chuyển đổi mã kí tự 1252 sang UTF-8
    def covert_unicode(self, txt):
        dicchar = self.load_DictChar()
        return re.sub(
            r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
            lambda x: dicchar[x.group()], txt) 

    # xóa phần không cần thiết trong văn bản
    def removeUnnecessaryPart(self, row):
        for i, line in enumerate(row.splitlines()):
            if re.match(r"^(PHẦN PHỤ LỤC|DANH MỤC PHỤ LỤC)", line) or re.match(r"^phụ lục", line.lower()) or re.match(r"^(\s|\t)*PHỤ LỤC", line):
                row = '\n'.join(row.splitlines()[:i])
                break     
        
        return row
    
    # đổi về chữ thường
    def text_lower(self, text):
        return text.lower()
    
    #  Tìm và thay thế các ký tự không in được trong text thành khoảng trắng
    def  replace_non_printable_chars(self, row):
        return  ''.join(char if char in string.printable or char.lower() in self.pattern else ' ' for char in row)
                
    def data_processing(self, row):
        row = self.covert_unicode(row)                          # Chuẩn hóa bảng mã kí 
        row = self.removeUnnecessaryPart(row)                   # Loại bỏ phần không cần thiết
        row = self.replace_non_printable_chars(row)             # Thay thế các chuỗi không in được thành khoảng trắng
        row = self.text_lower(row)                              # Đổi thành chữ thường
        
        result_line = []
        for line in row.splitlines():           
            if line != ' ' and line != '':
                line = line.replace('\n', '').replace('\t', '')                 # Loại \n và \t trong dòng
                line = self.Handling_Stuck_Syllables(line)                      # Tách âm tiết bị dính nhau
                line = ViTokenizer.tokenize(line).lstrip().rstrip()             # Tách câu, tách từ tiếng 
                result_line.append(line)
        row = "\n\n".join(result_line)

        row = row.replace(" / ", "/")
        row = row.replace(" - ", "-")
        row = self.highlight_object(row)

        return row

    # Chuẩn hóa y và i (vd: tỷ lệ -> tỉ lệ, bác sỹ -> bác sĩ)
    def chuan_hoa_y_i(self, word):
        set_yi ={1: 'yýỳỷỹỵ', 2: 'iíìỉĩị'}
        index_yi = None
        index_setyi = None
        for i in range(6):
            if set_yi[1][i] in word or set_yi[2][i] in word:
                index_yi = word.index(set_yi[1][i]) if set_yi[1][i] in word else word.index(set_yi[2][i])
                index_setyi = i
                break

        if index_yi is not None:
            if len(word) == 1 and word in set_yi[2]:
                word = set_yi[1][index_setyi]
            elif (len(word) > 1 and index_yi == len(word) - 1 and word[index_yi] in set_yi[1] 
            and word[:index_yi] in ['b','c','ch','d','đ','g','gh','h','k','kh','l','m','n','ng','ngh','nh','p','ph','r','s','t','th','tr','v','x']):
                word = word[:index_yi] + set_yi[2][index_setyi]

        return word

    # Xử lý các âm tiết bị lỗi dính nhau trong từng câu của văn bản
    def Handling_Stuck_Syllables(self, sentence):
        result_sentence = []
        sentence = self.split_speicalChar(sentence) # Tách các kí tự đặc biệt (ví dụ: 'luật.' -> 'luật .')
        for word in sentence.split():       # xét từng chữ
            word = self.standardize_Tone(word)  # Chuẩn hóa thanh điệu
            word = self.chuan_hoa_y_i(word)     # Chuẩn hóa y/i
            # Nếu từ không chứa kí tự, từ có nghĩa, là số ký hiệu hoặc là từ tiếng anh thì bỏ qua
            if (len(word) == 1
                    or any(char in word for char in self.specialCharset)
                    or any(char in word.lower() for char in ["f","j","w","z"])
                    or any(char in '0123456789' for char in word)
                    or self.words_en.check(word) 
                    or self.syllable_check(word) 
                    ):
                result_sentence.append(word)
                continue

            # Từ có nghĩa nhưng có 2 chữ giống nhau liên tiếp (vd: hoaang, luuật)
            word = self.double_char(word)
            if self.syllable_check(word):   # xét từ có nghĩa chưa
                result_sentence.append(word)
                continue

            check = True
            #Kiểm tra cặp từ có nghĩa trong từ điển
            for i in range(1, len(word)-1):
                text = word[:i] + " " + word[i:]
                if self.dictionary_check(text):
                    check = False
                    break
                
            # Nếu không có cặp từ có nghĩa
            if check:
                text = self.split_syllable(word)   # text = text (được tách ra)/ False (không được tách)

            # Xử lý các âm tiết bị lỗi dính nhau (các trường hợp chữ viết tắt và sai chính tả -> False)
            result_sentence.append(text if text else word) # nếu False thì từ được giữ nguyên

        return ' '.join(result_sentence)

    # Tách các kí tự đặc biệt 
    def split_speicalChar(self, text):  
        for i in self.specialCharset:
            if i in text:
                text = text.replace(i, f" {i} ")
        return text

    # Trả kết quả xử lý tách các âm tiết dính nhau
    def split_syllable(self, syllable):
        
        memories = []
        while syllable != '':
            text_len = len(syllable)    # Độ dài từ hiện tại
            memory = []                 # Chứa các từ có nghĩa 
            queue = ''                  # Cập nhật các từ

            for i in range(text_len):
                # Xử lý từ có nghĩa (vd: nóng -> memory = [nó, nón, nóng])
                queue += syllable[i]
                
                # Lọc ra cá kí tự đứng 1 mình, trừ các từ có thể
                if len(queue) == 1 and queue not in ['ả', 'ế', 'ô', 'ố', 'ổ', 'y', 'ý', 'ỷ', 'ủ']:
                    continue

                if self.syllable_check(queue) or self.dictionary_check(queue):
                    memory.append(queue)
            
            # Nếu memory có chứa các từ có nghĩa
            if memory:
                memories.append(memory)
                syllable = syllable[len(memory[-1]):]     # Loại bỏ từ có nghĩa trong chuỗi syllable và xét tiếp theo vòng lặp while
            # Nếu không
            else:
                # Nếu xét cả chuỗi không có từ nào hoặc chỉ 1 từ có nghĩa trong chuỗi -> False
                if memories and len(memories[0]) > 1:
                    # kiểm tra chuỗi từ trước có 2 từ trở lên thì ghép chữ cuối cùng của chuỗi từ đó [-1]  
                    for j in range(len(memories), 0, -1):
                        # Vd: hoặcđiago -> memories = [['cá', 'các', 'cách'],['i']] --- syllable = 'ệp'
                        if len(memories[j-1]) >= 2:
                            syllable = memories[j-1].pop(-1)[-1] + ''.join(c[-1] for c in memories[j:]) + syllable
                            memories = memories[:j]
                            break
                else:
                    return False
        
        return ' '.join(i[-1] for i in memories)

    # kiểm tra từ hoặc cụm từ có trong từ điển
    def dictionary_check(self, text):
        return text.lower() in self.words
    
    # kiểm tra âm tiết có trong từ điển
    def syllable_check(self, syllable):
        return syllable.lower() in self.syllables

    # xóa các kí tự lặp (trừ "o")
    def double_char(self, word):
        result = word[0]  # Khởi tạo result với ký tự đầu tiên của word
        for i in range(1, len(word)):  # Bắt đầu vòng lặp từ chỉ số 1
            if result[-1] == word[i] and word[i] != 'o':
                continue
            result += word[i]

        return result

    # Chuẩn hóa thanh điệu 
    def standardize_Tone(self, word):  
        if word != "gịa" and word != "quốc":
            for key, value in self.mapping.items():
                word = word.replace(key, value)
        return word
    
    # làm đối tượng nổi bật
    def highlight_object(self, sentence):
        for key in self.type_documents:
            sentence = sentence.replace(key.lower(), key)           
        return sentence

In [8]:
def entity_Extraction(A, sentence):
    
    regex_patterns = [r'([^;.<()>:]*)(Hiến_pháp) (?:(nước cộng_hoà xã_hội chủ_nghĩa việt_nam)|(\d+))([^;.<()>:]*)', "Bộ_luật", "Luật", "Pháp_lệnh", "Lệnh", "Quyết_định", 
                      "Nghị_định", "Nghị_quyết", "Nghị_quyết liên_tịch", "Thông_tư", "Thông_tư liên_tịch", "Chỉ_thị"]

    list_B = {
        'Thực thể tham chiếu': [],
        'Nội dung trước thực thể được tham chiếu': [],
        'Thực thể được tham chiếu': [],
        'Nội dung sau thực thể được tham chiếu': []
    }
    for key in regex_patterns:
        if key == regex_patterns[0]:
            matches = re.findall(key, sentence)
        elif (key == 'Nghị_định' or key == 'Thông_tư') and 'liên_tịch' in sentence[sentence.find(key) + 9 : sentence.find(key) + 20]:
                continue
        else:
            pattern = r'([^;.<()>:]*)'+rf'({key})' + r' ((?:(?!ngày|số|này|tại|theo|thì|và|[A-ZĐ])\w+[ _])+)?(số)?\s?(\d+?[\w]+(?:(?:/|-)+(?:\d|\w)+)+)?\s?(ngày \d{1,2}(?: tháng |[-]|[/])\d{1,2}(?: năm |[-]|[/])\d{4})?([^;.<()>:]*)'
            matches = re.findall(pattern, sentence)

        list_entity = list(set(matches))
        if list_entity:     
            for entity in list_entity:
                start_entity = entity[0]
                end_entity = entity[-1]
                entity_tuple = tuple(filter(lambda x: x != '', entity[1:-1]))
                B = ' '. join(entity_tuple).replace('  ', ' ')
                
                if len(entity_tuple) < 2 or B in A:
                    continue
                
                list_B['Thực thể tham chiếu'].append(A)
                list_B['Nội dung trước thực thể được tham chiếu'].append(start_entity)
                list_B['Thực thể được tham chiếu'].append(B)
                list_B['Nội dung sau thực thể được tham chiếu'].append(end_entity)


    df = pd.DataFrame(list_B) 

    return df


vb = tvpl_function()
A = 'Luật Bảo hiểm xã hội'
A = vb.data_processing(A)
B = 'Nam đủ 55 tuổi , nữ đủ 50 tuổi , trừ trường_hợp Luật sĩ_quan quân_đội nhân_dân Việt_Nam, Luật công_an nhân_dân, Luật cơ_yếu có quy_định khác'
text = vb.data_processing(B)
X = entity_Extraction(A, text)
X['Text'] = X.apply(lambda d: f"{d['Thực thể tham chiếu']} {d['Nội dung trước thực thể được tham chiếu']} {d['Thực thể được tham chiếu']} {d['Nội dung sau thực thể được tham chiếu']}".strip(), axis=1)
X

,Thực thể tham chiếu,Nội dung trước thực thể được tham chiếu,Thực thể được tham chiếu,Nội dung sau thực thể được tham chiếu,Text
0,Luật bảo_hiểm xã_hội,"nam đủ 55 tuổi , nữ đủ 50 tuổi , trừ trường _ ...",Luật cơ _ yếu có quy _ định,khác,"Luật bảo_hiểm xã_hội nam đủ 55 tuổi , nữ đủ 50..."


In [5]:
A = 'thông tư số 88/2015/tt-btc ngày 09 tháng 06 năm 2015 của bộ tài chính'
A = vb.data_processing(A)

print(A)

Thông_tư số 88/2015/t-btc ngày 09 tháng 06 năm 2015 của bộ tài_chính
